## Import the scene graph file

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd

# Authenticate
drive = None
def authenticate():
  global drive
  
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

#Download files
def downloadFiles(fileIds):
  authenticate()
  
  for fileId in fileIds:    
    
    downloaded = drive.CreateFile({"id": fileId[1]})
    downloaded.GetContentFile(fileId[0])

In [ ]:
#Do not downloading training and validation dataset at same time 
try:
  _ = open("docbank_full_feature_sg_train.pkl", "r")
except:
  downloadFiles([["docbank_full_feature_sg_train.pkl", "1-0xDvqXhEOC-amZ83yy8V4a03uqa6Vvg"]])

In [ ]:
# Testing Dataset
#1-1ycOHYAyE4MrKoBbvhN0pU-6jvU5wjf

In [ ]:
# downloading the scene graph file
import pickle
with open('/content/drive/MyDrive/Docbank/docbank_sg_test_5000_density_gap_rel_lstmhidden.pkl', 'rb') as f:
    old_train_list_dict=pickle.load(f)

In [ ]:
print(old_train_list_dict.keys())
print(old_train_list_dict[1]['objects']['6'].keys())

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [ ]:
del old_train_list_dict[1945]
del old_train_list_dict[5859]
del old_train_list_dict[10553]
del old_train_list_dict[15273]
del old_train_list_dict[15916]
del old_train_list_dict[16701]
del old_train_list_dict[18484]
del old_train_list_dict[18700]
del old_train_list_dict[18706]
del old_train_list_dict[19573]
del old_train_list_dict[19729]

In [ ]:
train_list_dict = old_train_list_dict

In [ ]:
train_list_dict = {}
i = 0
for img in old_train_list_dict:
  if i>=10000:
    train_list_dict[img] = old_train_list_dict[img]
  i += 1

test part

In [ ]:
# downloading the scene graph file
import pickle
with open('/content/drive/MyDrive/Docbank/docbank_sg_test_5000_density_gap_rel_lstmhidden.pkl', 'rb') as f:
    train_list_dict=pickle.load(f)

In [ ]:
del train_list_dict[21772]
del train_list_dict[22182]
del train_list_dict[23833]
del train_list_dict[23865]
del train_list_dict[23866]
del train_list_dict[23867]

## Define some pre-processing function

### Set **k** nearest neighbour for distance base weighted graph
This is used to generate visual feature and text density based graph embedding, to generate a new key for object dictionary named 'near gap'.

In [ ]:
# selected top k gap distance features
def top_k_nearest(gap_dict, k):
  new_gap_dict = {}
  try:
    list1= sorted(gap_dict.values())
  except:
    print(gap_dict)
  top_k_value = list1[:k]
  for obj in gap_dict:
    if gap_dict[obj] in top_k_value:
      new_gap_dict[obj] = gap_dict[obj]
  return new_gap_dict

In [ ]:
# default k is 2
for img in train_list_dict:
  for obj in train_list_dict[img]['objects']:
    new_gap_dict = top_k_nearest(train_list_dict[img]['objects'][obj]['gap'],2)
    train_list_dict[img]['objects'][obj]['near_gap'] = new_gap_dict

### Others

In [ ]:
# according to id to search text
bbox_id_text = {}
for l in train_list_dict:
  for obj in train_list_dict[l]['objects']:
    id = train_list_dict[l]['objects'][obj]['id']
    bbox_id_text[id] = {}
    bbox_id_text[id]['text'] = train_list_dict[l]['objects'][obj]['text']


In [ ]:
for l in eval_list_dict:
  for obj in eval_list_dict[l]['objects']:
    id = eval_list_dict[l]['objects'][obj]['id']
    bbox_id_text[id] = {}
    bbox_id_text[id]['text'] = eval_list_dict[l]['objects'][obj]['text']

In [ ]:
# according to id to search object id in a document
def globalid_to_localid(id):
  for l in train_list_dict:
    for obj in train_list_dict[l]['objects']:
      if id == train_list_dict[l]['objects'][obj]['id']:
        return obj
  for l in eval_list_dict:
    for obj in eval_list_dict[l]['objects']:
      if id == eval_list_dict[l]['objects'][obj]['id']:
        return obj

In [ ]:
id_density_dict = {}
for l in train_list_dict:
  id_density_dict[l] = {}
  for obj in train_list_dict[l]['objects']:
    id_density_dict[l][obj] = train_list_dict[l]['objects'][obj]['parsing2']

In [ ]:
print(id_density_dict[0])

In [ ]:
# this dictionary used to transfer object id to text_density/text_number/char_density/char_number/visual_embedding/
for l in eval_list_dict:
  id_density_dict[l] = {}
  for obj in eval_list_dict[l]['objects']:
    id_density_dict[l][obj] = eval_list_dict[l]['objects'][obj]['parsing_level1']


In [ ]:
import math
import torch
# one dimensional feature embedding
def positionalencoding1d(d_model, feature_list):
    """
    :param d_model: dimension of the model
    :param feature_list: length of positions
    :return: length*d_model position matrix
    """
    if d_model % 2 != 0:
        raise ValueError("Cannot use sin/cos positional encoding with "
                         "odd dim (got dim={:d})".format(d_model))
    pe = torch.zeros(len(feature_list), d_model)
    feats = torch.tensor(feature_list).unsqueeze(1)
    div_term = torch.exp((torch.arange(0, d_model, 2, dtype=torch.float) *
                         -(math.log(10000.0) / d_model)))
    pe[:, 0::2] = torch.sin(feats.float() * div_term)
    pe[:, 1::2] = torch.cos(feats.float() * div_term)
    pe = pe.tolist()
    return pe




## Extracting Information from training and evaluation json file

In [ ]:
train_list_dict[22824]['objects']['0'].keys()

In [ ]:
# Generate local graph based training dataset objects and relations list
new_train_list_dict = {}
for l in train_list_dict:
  tem_dic = {}
  tem_dic['objects'] = []
  tem_dic['relationships'] = []
  
  for obj in train_list_dict[l]['objects']:
    tem_dic['objects'].append(obj)
    for rel in train_list_dict[l]['objects'][obj]['relations']:
      tem_rel = [obj]
      obj2_id = rel[1]
      tem_rel.append(obj2_id)
      name = rel[0]
      tem_rel.append(name)
      tem_dic['relationships'].append(tem_rel)
  new_train_list_dict[l] = tem_dic

In [ ]:
# Generate local graph based validation dataset objects and relations list
new_eval_list_dict = {}
for l in eval_list_dict:
  tem_dic = {}
  tem_dic['objects'] = []
  tem_dic['relationships'] = []

  for obj in eval_list_dict[l]['objects']:
    tem_dic['objects'].append(obj)
    for rel in eval_list_dict[l]['objects'][obj]['relations']:
      tem_rel = [obj]
      obj2_id = eval_list_dict[l]['objects'][obj]['relations'][rel]['object']
      obj2_id= globalid_to_localid(obj2_id)
      tem_rel.append(obj2_id)
      label1 = eval_list_dict[l]['objects'][obj]['category']
      label2 = eval_list_dict[l]['objects'][obj2_id]['category']
      rel_name = relation_naming(label1,label2)
      tem_rel.append(rel_name)
      tem_dic['relationships'].append(tem_rel)
  new_eval_list_dict[l] = tem_dic

In [ ]:
#print(train_list_dict['282944']['objects']['4'])
print(train_list_dict[23295]['objects']['0']['category'])
print(new_train_list_dict[23295]['relationships'])

In [ ]:
#Using for searching real node size
num_obj_dict = {}
for l in new_train_list_dict:
    num_obj_dict[l] = len(new_train_list_dict[l]['objects'])


In [ ]:
for l in new_eval_list_dict:
    num_obj_dict[l] = len(new_eval_list_dict[l]['objects'])

In [ ]:
print(num_obj_dict)

{'282944': 9, '285236': 3, '284513': 6, '285613': 16, '282966': 11, '254597': 10, '285483': 12, '282973': 5, '285257': 8, '285511': 13, '286052': 3, '286059': 6, '282987': 10, '286185': 12, '282994': 10, '286192': 12, '285299': 4, '284597': 4, '285541': 10, '286199': 10, '286207': 7, '284604': 19, '286214': 4, '284919': 14, '285549': 9, '284940': 7, '284625': 10, '285124': 8, '284956': 7, '284639': 11, '283009': 9, '283016': 6, '285592': 13, '284667': 19, '285669': 7, '286417': 3, '283038': 7, '286424': 3, '286431': 3, '283052': 8, '286438': 4, '286445': 7, '286508': 10, '285399': 7, '286452': 5, '287671': 14, '287678': 14, '287493': 7, '285350': 4, '287451': 15, '288065': 11, '288826': 13, '287699': 17, '288791': 25, '288742': 4, '288749': 10, '289026': 11, '286566': 4, '285166': 10, '289314': 9, '290010': 11, '287734': 9, '289892': 10, '290038': 13, '289836': 11, '287741': 12, '289899': 12, '289913': 11, '289921': 17, '288924': 11, '288931': 13, '290685': 6, '288946': 6, '290755': 6,

In [ ]:
# useless code only rename
list_dict_train={}
for l in new_train_list_dict:
    list_dict_train[l] = new_train_list_dict[l]


In [ ]:
list_dict_test = {}
for l in new_eval_list_dict:
    list_dict_test[l] = new_eval_list_dict[l]

## Training and validation dataframe generation

In [ ]:
#generate the training and labeling information for each image
#this information can be used to transfer object id to corresponding visual, text density or other kinds of features.
density_list_train = []
label_list_train = []
img_list_train = []
label_dict_train = {}
for l in train_list_dict:
  label_dict_train[l] = []
  for obj in train_list_dict[l]['objects']:
    density_list_train.append(obj)
    label_list_train.append(train_list_dict[l]['objects'][obj]['category'])
    img_list_train.append(l)
    label_dict_train[l].append(train_list_dict[l]['objects'][obj]['category'])
  for obj in range(num_obj_dict[l],177):
    density_list_train.append(str(obj))
    label_list_train.append(str(-1))
    img_list_train.append(l)
    label_dict_train[l].append(str(-1))

In [ ]:
# generating evalutaion labeling informaiton dictionary
density_list_eval = []
label_list_eval = []
img_list_eval = []
label_dict_eval = {}
for l in eval_list_dict:
  label_dict_eval[l] = []
  for obj in eval_list_dict[l]['objects']:
    density_list_eval.append(obj)
    label_list_eval.append(eval_list_dict[l]['objects'][obj]['category'])
    img_list_eval.append(l)
    label_dict_eval[l].append(eval_list_dict[l]['objects'][obj]['relations'])
  for obj in range(num_obj_dict[l],181):
    density_list_eval.append(str(obj))
    label_list_eval.append(str(-1))
    img_list_eval.append(l)
    label_dict_eval[l].append(str(-1))

In [ ]:
# this dictionary used to transfer object id to text density/visual feature/ label
'''
id_density_dict = {}
for l in eval_list_dict:
  id_density_dict[l] = {}
  for obj in eval_list_dict[l]['objects']:
    id_density_dict[l][obj] = eval_list_dict[l]['objects'][obj]['category']
for l in train_list_dict:
  id_density_dict[l] = {}
  for obj in train_list_dict[l]['objects']:
    id_density_dict[l][obj] = train_list_dict[l]['objects'][obj]['category']
'''

In [ ]:
# this dictionary used to transfer object id to bert
id_bert_dict = {}
j = 0
for l in eval_list_dict:
  id_bert_dict[l] = {}
  for obj in eval_list_dict[l]['objects']:
    id_bert_dict[l][obj] = bert_list_test[j]
    j += 1

j = 0
for l in train_list_dict:
  id_bert_dict[l] = {}
  for obj in train_list_dict[l]['objects']:
    id_bert_dict[l][obj] = bert_list_train[j]
    j += 1

In [ ]:
from pandas import DataFrame
df_train = DataFrame(density_list_train,columns=['density'])
df_train['label'] = label_list_train
df_train['image'] = img_list_train

In [ ]:
label_list_train = df_train['label'].fillna('other').tolist()
df_train_clean = df_train[df_train['label'].notnull()][['density','label','image']]

In [ ]:
label_list_train = df_train_clean['label'].to_list()
id_list_train = df_train_clean['density'].to_list()
img_list_train = df_train_clean['image'].to_list()


In [ ]:
obj_list_train = []
for i in range(len(id_list_train)):
  tem = []
  tem = [id_list_train[i],img_list_train[i]]
  obj_list_train.append(tem)

In [ ]:
print(list_dict_train)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Convert label into one-hot

In [ ]:
'''
labels_one_hot_train = {}
for f in label_dict_train:
  labels_one_hot_train[f] = []
  for l in label_dict_train[f]:
    to_add = [0]*181
    for j in l:
      try:
        id = l[j]['object']
        link = int(str(id))
        to_add[link] = 1
      except: pass
    labels_one_hot_train[f].append(to_add)
'''

In [ ]:
'''
labels_one_hot_eval = {}
for f in label_dict_eval:
  labels_one_hot_eval[f] = []
  for l in label_dict_eval[f]:
    to_add = [0]*181
    for j in l:
      try:
        id = l[j]['object']
        link = int(str(id))
        to_add[link] = 1
      except: pass
    labels_one_hot_eval[f].append(to_add)
'''

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
num_class = len(list(set(label_list_train)))
lEnc = LabelEncoder()
lEnc.fit(np.unique(list(set(label_list_train))))
labels_one_hot_train = {}
for f in label_dict_train:
  num_labels = lEnc.transform(label_dict_train[f])
  labels_one_hot_train[f] = []
  for l in num_labels:
    to_add = [0]*num_class
    to_add[l]=1
    labels_one_hot_train[f].append(to_add)


In [ ]:
labels_one_hot_eval = {}
for f in label_dict_eval:
  num_labels = lEnc.transform(label_dict_eval[f])
  labels_one_hot_eval[f] = []
  for l in num_labels:
    to_add = [0]*num_class
    to_add[l]=1
    labels_one_hot_eval[f].append(to_add)

## Build Graph

In [ ]:
print(new_train_list_dict.keys())

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [ ]:
node_lists_train = {}
for l in new_train_list_dict:
  node_lists_train[l] = []
  for obj in new_train_list_dict[l]['objects']:
    node_lists_train[l].append(obj)

In [ ]:
node_lists_eval = {}
for l in new_eval_list_dict:
  node_lists_eval[l] = []
  for obj in new_eval_list_dict[l]['objects']:
    node_lists_eval[l].append(obj)

In [ ]:
print(node_lists_train)

{0: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'], 1: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 2: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11'], 3: ['0', '1', '2', '3', '4', '5', '6', '7', '8'], 4: ['0', '1', '2', '3', '4', '5'], 5: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'], 6: ['0', '1', '2', '3', '4', '5'], 7: ['0', '1', '2', '3', '4', '5', '6'], 8: ['0', '1', '2', '3', '4', '5'], 9: ['0', '1', '2', '3', '4', '5', '6'], 10: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32'], 11: ['0', '1', '2', '3', '4', '5'], 12: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20'], 13: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18',

In [ ]:
# build local graph
object_graph_dict_train = {}
for l in new_train_list_dict:
  object_graph_dict_train[l] = {}
  for obj in new_train_list_dict[l]['objects']:
    object_graph_dict_train[l][obj] = {}
    object_graph_dict_train[l][obj]['obj2'] = []


In [ ]:
object_graph_dict_eval = {}
for l in new_eval_list_dict:
  object_graph_dict_eval[l] = {}
  for obj in new_eval_list_dict[l]['objects']:
    object_graph_dict_eval[l][obj] = {}
    object_graph_dict_eval[l][obj]['obj2'] = []

In [ ]:
print(len(list_dict_train))

19989


In [ ]:
print(list_dict_test.keys())
print(list_dict_test['83594639'])

In [ ]:
#fill the empty graph
for sg in list_dict_train:
    obj_list = list_dict_train[sg]['objects']
    for rel_item in list_dict_train[sg]['relationships']:
        obj1 = str(rel_item[0])
        obj2 = str(rel_item[1])
        object_graph_dict_train[sg][rel_item[0]]['obj2'].append(obj2)

In [ ]:
#fill the empty graph
for sg in list_dict_test:
    obj_list = list_dict_test[sg]['objects']
    for rel_item in list_dict_test[sg]['relationships']:
        obj1 = str(rel_item[0])
        obj2 = str(rel_item[1])
        object_graph_dict_eval[sg][rel_item[0]]['obj2'].append(obj2)

In [ ]:
node_size_train = []
for f in object_graph_dict_train:
  l = object_graph_dict_train[f].keys()
  node_size_train.append(len(l))

In [ ]:
node_size_eval = []
for f in object_graph_dict_eval:
  l = object_graph_dict_eval[f].keys()
  node_size_eval.append(len(l))

In [ ]:

#padding methods
#weight for padding is -1
#node id if for padding nodes is -1
max_len = 177
col_train = {}
row_train = {}
weight_train = {}
for f in object_graph_dict_train:
  col_train[f] = []
  row_train[f] = []
  weight_train[f] = []
  for obj in object_graph_dict_train[f]:
    obj_rel_list = object_graph_dict_train[f][obj]['obj2']
    for obj2 in object_graph_dict_train[f]:
      if obj2 in obj_rel_list:
        weight_train[f].append(1.0)
      else:
        weight_train[f].append(0.0)
      row_train[f].append(obj)
      col_train[f].append(obj2)


In [ ]:
len(weight_train[f])
print(train_list_dict[19999]['objects'].keys())

dict_keys(['0', '1', '2', '3'])


In [ ]:

#padding methods
#weight for padding is -1
#node id if for padding nodes is -1
max_len = 181
col_eval = {}
row_eval = {}
weight_eval = {}
for f in object_graph_dict_eval:
  col_eval[f] = []
  row_eval[f] = []
  weight_eval[f] = []
  for obj in object_graph_dict_eval[f]:
    obj_rel_list = object_graph_dict_eval[f][obj]['obj2']
    for i in range(len(obj_rel_list)):
      try:
        weight_eval[f].append(1)
      except:
        weight_eval[f].append(0.0)
      row_eval[f].append(obj)
      col_eval[f].append(obj_rel_list[i])

In [ ]:
#PROBLEMS
# how to determine the weight to the node itself
# how to padding the node

In [ ]:
import scipy.sparse as sp
node_size = 177
adj_train = {}
for f in weight_train:
  adj_train[f] = sp.csr_matrix((weight_train[f], (row_train[f], col_train[f])), shape=(node_size, node_size))

In [ ]:
import scipy.sparse as sp
node_size = 177
adj_eval = {}
for f in weight_eval:
  adj_eval[f] = sp.csr_matrix((weight_eval[f], (row_eval[f], col_eval[f])), shape=(node_size, node_size))

##Build Model

In [ ]:
# from inits import *
import tensorflow.compat.v1 as tf

flags = tf.app.flags
FLAGS = flags.FLAGS

# global unique layer ID dictionary for layer name assignment
_LAYER_UIDS = {}


def get_layer_uid(layer_name=''):
    """Helper function, assigns unique layer IDs."""
    if layer_name not in _LAYER_UIDS:
        _LAYER_UIDS[layer_name] = 1
        return 1
    else:
        _LAYER_UIDS[layer_name] += 1
        return _LAYER_UIDS[layer_name]


def sparse_dropout(x, keep_prob, noise_shape):
    """Dropout for sparse tensors."""
    random_tensor = keep_prob
    random_tensor += tf.random_uniform(noise_shape)
    dropout_mask = tf.cast(tf.floor(random_tensor), dtype=tf.bool)
    pre_out = tf.sparse_retain(x, dropout_mask)
    return pre_out * (1./keep_prob)


def dot(x, y, sparse=False):
    """Wrapper for tf.matmul (sparse vs dense)."""
    if sparse:
        res = tf.sparse_tensor_dense_matmul(x, y)
    else:
        res = tf.matmul(x, y)
    return res


class Layer(object):
    """Base layer class. Defines basic API for all layer objects.
    Implementation inspired by keras (http://keras.io).

    # Properties
        name: String, defines the variable scope of the layer.
        logging: Boolean, switches Tensorflow histogram logging on/off

    # Methods
        _call(inputs): Defines computation graph of layer
            (i.e. takes input, returns output)
        __call__(inputs): Wrapper for _call()
        _log_vars(): Log all variables
    """

    def __init__(self, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            layer = self.__class__.__name__.lower()
            name = layer + '_' + str(get_layer_uid(layer))
        self.name = name
        self.vars = {}
        logging = kwargs.get('logging', False)
        self.logging = logging
        self.sparse_inputs = False

    def _call(self, inputs):
        return inputs

    def __call__(self, inputs):
        with tf.name_scope(self.name):
            if self.logging and not self.sparse_inputs:
                tf.summary.histogram(self.name + '/inputs', inputs)
            outputs = self._call(inputs)
            if self.logging:
                tf.summary.histogram(self.name + '/outputs', outputs)
            return outputs

    def _log_vars(self):
        for var in self.vars:
            tf.summary.histogram(self.name + '/vars/' + var, self.vars[var])



class GraphConvolution(Layer):
    """Graph convolution layer."""
    def __init__(self, input_dim, output_dim, placeholders, dropout=0.,
                 sparse_inputs=False, act=tf.nn.relu, bias=False,
                 featureless=False, **kwargs):
        super(GraphConvolution, self).__init__(**kwargs)

        if dropout:
            self.dropout = placeholders['dropout']
        else:
            self.dropout = 0.

        self.act = act
        self.support = placeholders['support']
        self.sparse_inputs = sparse_inputs
        self.featureless = featureless
        self.bias = bias

        # helper variable for sparse dropout
        self.num_features_nonzero = placeholders['num_features_nonzero']

        with tf.variable_scope(self.name + '_vars'):
            for i in range(len(self.support)):
                self.vars['weights_' + str(i)] = glorot([input_dim, output_dim],
                                                        name='weights_' + str(i))
            if self.bias:
                self.vars['bias'] = zeros([output_dim], name='bias')

        if self.logging:
            self._log_vars()

    def _call(self, inputs):
        x = inputs

        # dropout
        if self.sparse_inputs:
            x = sparse_dropout(x, 1-self.dropout, self.num_features_nonzero)
        else:
            x = tf.nn.dropout(x, 1-self.dropout)

        # convolve
        supports = list()
        for i in range(len(self.support)):
            if not self.featureless:
                pre_sup = dot(x, self.vars['weights_' + str(i)],
                              sparse=self.sparse_inputs)
            else:
                pre_sup = self.vars['weights_' + str(i)]            
            support = dot(self.support[i], pre_sup, sparse=True)
            supports.append(support)
        output = tf.add_n(supports)

        # bias
        if self.bias:
            output += self.vars['bias']
			
		
        self.embedding = output #output
        return self.act(output)


In [ ]:
# from metrics import *
import tensorflow.compat.v1 as tf

flags = tf.app.flags
FLAGS = flags.FLAGS

def glorot(shape, name=None):
    """Glorot & Bengio (AISTATS 2010) init."""
    init_range = np.sqrt(6.0/(shape[0]+shape[1]))
    initial = tf.random_uniform(shape, minval=-init_range, maxval=init_range, dtype=tf.float32)
    return tf.Variable(initial, name=name)

class Model(object):
    def __init__(self, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            name = self.__class__.__name__.lower()
        self.name = name

        logging = kwargs.get('logging', False)
        self.logging = logging

        self.vars = {}
        self.placeholders = {}

        self.layers = []
        self.activations = []

        self.inputs = None
        self.outputs = None

        self.loss = 0
        self.accuracy = 0
        self.optimizer = None
        self.opt_op = None

    def _build(self):
        raise NotImplementedError

    def build(self):
        """ Wrapper for _build() """
        with tf.variable_scope(self.name):
            self._build()

        # Build sequential layer model
        self.activations.append(self.inputs)
        for layer in self.layers:
            hidden = layer(self.activations[-1])
            self.activations.append(hidden)
        self.outputs = self.activations[-1]

        # Store model variables for easy access
        variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.name)
        self.vars = {var.name: var for var in variables}

        # Build metrics
        self._loss()
        self._accuracy()

        self.opt_op = self.optimizer.minimize(self.loss)

    def predict(self):
        pass

    def _loss(self):
        raise NotImplementedError

    def _accuracy(self):
        raise NotImplementedError

    def save(self, sess=None):
        if not sess:
            raise AttributeError("TensorFlow session not provided.")
        saver = tf.train.Saver(self.vars)
        save_path = saver.save(sess, "tmp/%s.ckpt" % self.name)
        print("Model saved in file: %s" % save_path)

    def load(self, sess=None):
        if not sess:
            raise AttributeError("TensorFlow session not provided.")
        saver = tf.train.Saver(self.vars)
        save_path = "tmp/%s.ckpt" % self.name
        saver.restore(sess, save_path)
        print("Model restored from file: %s" % save_path)

In [ ]:
class GCN(Model):
    def __init__(self, placeholders, input_dim, **kwargs):
        super(GCN, self).__init__(**kwargs)

        self.inputs = placeholders['features']
        self.input_dim = input_dim
        # self.input_dim = self.inputs.get_shape().as_list()[1]  # To be supported in future Tensorflow versions
        self.output_dim = placeholders['labels'].get_shape().as_list()[1]
        self.placeholders = placeholders

        self.optimizer = tf.train.AdamOptimizer(learning_rate=FLAGS.learning_rate)

        self.build()

    def _loss(self):
        # Weight decay loss
        for var in self.layers[0].vars.values():
            self.loss += FLAGS.weight_decay * tf.nn.l2_loss(var)

        # Cross entropy error
        self.loss += masked_softmax_cross_entropy(self.outputs, self.placeholders['labels'],
                                                  self.placeholders['labels_mask'])

    def _accuracy(self):
        self.accuracy = masked_accuracy(self.outputs, self.placeholders['labels'],
                                        self.placeholders['labels_mask'])
        self.pred = tf.argmax(self.outputs, 1)
        self.labels = tf.argmax(self.placeholders['labels'], 1)

    def _build(self):

        self.layers.append(GraphConvolution(input_dim=self.input_dim,
                                            output_dim=FLAGS.hidden1,
                                            placeholders=self.placeholders,
                                            act=tf.nn.relu,
                                            dropout=True,
                                            featureless=False,
                                            sparse_inputs=True,
                                            logging=self.logging))
        
        self.layers.append(GraphConvolution(input_dim=FLAGS.hidden1,
                                            output_dim=self.output_dim,
                                            placeholders=self.placeholders,
                                            act=lambda x: x, #
                                            dropout=True,
                                            logging=self.logging))

    def predict(self):
        return tf.nn.sigmoid(self.outputs)

In [ ]:
def masked_softmax_cross_entropy(preds, labels, mask):
    """Softmax cross-entropy loss with masking."""
    print(preds)
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=labels)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    loss *= mask
    return tf.reduce_mean(loss)


def masked_accuracy(preds, labels, mask):
    """Accuracy with masking."""
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(labels, 1))

    accuracy_all = tf.cast(correct_prediction, tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    accuracy_all *= mask
    return tf.reduce_mean(accuracy_all)

## Model Setting

In [ ]:
from __future__ import division
from __future__ import print_function

import time
import tensorflow.compat.v1 as tf

from sklearn import metrics
import random
import os
import sys


# # Set random seed
# seed = random.randint(1, 200)
# np.random.seed(seed)
# tf.set_random_seed(seed)


# Settings
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"

flags = tf.app.flags
FLAGS = flags.FLAGS
for name in list(flags.FLAGS):
      delattr(flags.FLAGS,name)
flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_string('dataset', 'sencegraph', 'Dataset string.')
# 'gcn', 'gcn_cheby', 'dense'
flags.DEFINE_string('model', 'gcn', 'Model string.')
flags.DEFINE_float('learning_rate', 0.001, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 10, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 768, 'Number of units in hidden layer 1.')
flags.DEFINE_float('dropout', 0.5, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('weight_decay', 0,
                   'Weight for L2 loss on embedding matrix.')  # 5e-4
flags.DEFINE_integer('early_stopping', 20,
                     'Tolerance for early stopping (# of epochs).')
flags.DEFINE_integer('max_degree', 3, 'Maximum Chebyshev polynomial degree.')

### Bert CLS as Node Feature

In [ ]:
# wrong code for global level 
# transfer from node id to features
import scipy.sparse as sp
import numpy as np
features_train = {}
for l in node_lists_train:
  features_train[l] = []
  for node in node_lists_train[l]:
    feature = np.array([0.0]*768)
    feature += id_bert_dict[l][str(node)]
    features_train[l].append(feature)
  for i in range(len(node_lists_train[l]),181):
    feature = np.array([0.0]*768)
    features_train[l].append(feature)
  features_train[l]=sp.csr_matrix(features_train[l])

In [ ]:
import scipy.sparse as sp
features_eval = {}
for l in node_lists_eval:
  features_eval[l] = []
  for node in node_lists_eval[l]:
    feature = np.array([0.0]*768)
    feature += id_bert_dict[l][str(node)]
    features_eval[l].append(feature)
  for i in range(len(node_lists_eval[l]),181):
    feature = np.array([0.0]*768)
    features_eval[l].append(feature)
  features_eval[l]=sp.csr_matrix(features_eval[l])

### Other features

In [ ]:
# wrong code for global level 
# transfer from node id to features
import scipy.sparse as sp
features_train = {}
for l in node_lists_train:
  features_train[l] = []
  for node in node_lists_train[l]:
    feature = np.array([0.0]*768)
    feature += np.array((id_density_dict[l][node]))
    features_train[l].append(feature)
  for i in range(len(node_lists_train[l]),177):
    feature = np.array([0.0]*768)
    features_train[l].append(feature)
  features_train[l]=sp.csr_matrix(features_train[l])

In [ ]:
# wrong code for global level 
# transfer from node id to features
import scipy.sparse as sp
features_train = {}
for l in node_lists_train:
  features_train[l] = []
  for node in node_lists_train[l]:
    feature = np.array([0.0]*2048)
    feature += id_density_dict[l][str(node)]
    features_train[l].append(feature)
  for i in range(len(node_lists_train[l]),85):
    feature = np.array([0.0]*2048)
    features_train[l].append(feature)
  features_train[l]=sp.csr_matrix(features_train[l])

In [ ]:
'''
import scipy.sparse as sp
features = []
for tokens in tokenize_nodes:
  feature = np.array([0.0]*2048)
  for token in tokens:
    try:
      feature += id_density_dict[int(tokens[0])]
    except:
      pass
  features.append(feature)
features= sp.csr_matrix(features)
'''

In [ ]:
print(len(features_train))

### Sparse to tuple

In [ ]:
def sparse_to_tuple(sparse_mx):
    """Convert sparse matrix to tuple representation."""
    def to_tuple(mx):
        if not sp.isspmatrix_coo(mx):
            mx = mx.tocoo()
        coords = np.vstack((mx.row, mx.col)).transpose()
        values = mx.data
        shape = mx.shape
        return coords, values, shape
    if isinstance(sparse_mx, list):
        for i in range(len(sparse_mx)):
            sparse_mx[i] = to_tuple(sparse_mx[i])
    else:
        sparse_mx = to_tuple(sparse_mx)
    return sparse_mx


def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return sparse_to_tuple(features)
for l in features_train:
  features_train[l] = preprocess_features(features_train[l])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21:

In [ ]:
for l in features_eval:
  features_eval[l] = preprocess_features(features_eval[l])

In [ ]:
print(adj_train['91391310'][0].shape)

## input setting

In [ ]:
#define input dictionary
# num_obj_dict was defined to record the number of object in each document page
#
y_train = {}
train_mask = {}
for f in new_train_list_dict:
  y_train[f] = np.array([[0]*177]*adj_train[f].shape[0])
  y_train[f] = np.array(labels_one_hot_train[f])
  train_mask[f] = [False]*adj_train[f].shape[0]
  train_mask[f][:num_obj_dict[f]] = [True]*num_obj_dict[f]


In [ ]:
y_val = {}
val_mask = {}
for f in new_eval_list_dict:
  y_val[f] = np.array([[0]*177]*adj_eval[f].shape[0])
  y_val[f] = np.array(labels_one_hot_eval[f])
  val_mask[f] = [False]*adj_eval[f].shape[0]
  val_mask[f][:num_obj_dict[f]] = [True]*num_obj_dict[f]

In [ ]:
def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()

def preprocess_adj(adj):
    """Preprocessing of adjacency matrix for simple GCN model and conversion to tuple representation."""
    adj_normalized = normalize_adj(adj + sp.eye(adj.shape[0]))
    return sparse_to_tuple(adj_normalized)
support_train = {}
for f in adj_train:
  support_train[f] = [preprocess_adj(adj_train[f])]

In [ ]:
support_eval = {}
for f in adj_eval:
  support_eval[f] = [preprocess_adj(adj_eval[f])]

In [ ]:
# just change the number of support gpu
num_supports = 1
tf.compat.v1.disable_eager_execution()
# Define placeholders
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    'features': tf.sparse_placeholder(tf.float32, shape = (14,768)),
    'labels': tf.placeholder(tf.float32, shape=(None, 14)),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0.5, shape=()),
    # helper variable for sparse dropout
    'num_features_nonzero': tf.placeholder(tf.int32)
}

# Create model
#print(features[2][1])
# placeholders: number of GCN

model = GCN(placeholders, input_dim=768, logging=True)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Tensor("graphconvolution_2/SparseTensorDenseMatMul/SparseTensorDenseMatMul:0", shape=(None, 14), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [ ]:
# Initialize session
session_conf = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config=session_conf)


# Define model evaluation function
def evaluate(features, support, labels, mask, placeholders):
    feed_dict_val = construct_feed_dict(
        features, support, labels, mask, placeholders)
    outs_val = sess.run([model.loss, model.layers[0].embedding, model.layers[1].embedding], feed_dict=feed_dict_val)
    return outs_val[0], outs_val[1], outs_val[2]


# Init variables
sess.run(tf.global_variables_initializer())

cost_val = []



In [ ]:
def construct_feed_dict(features, support, labels, labels_mask, placeholders):
    """Construct feed dictionary."""
    feed_dict = dict()
    feed_dict.update({placeholders['labels']: labels})
    feed_dict.update({placeholders['labels_mask']: labels_mask})
    feed_dict.update({placeholders['features']: features})
    feed_dict.update({placeholders['support'][i]: support[i]
                      for i in range(len(support))})
    feed_dict.update({placeholders['num_features_nonzero']: features[1].shape})
    return feed_dict

## Training

In [ ]:
# Train model
for epoch in range(FLAGS.epochs):
    print(epoch)
    loss = 0
    for f in features_train:
        
        feed_dict = construct_feed_dict(
            features_train[f], support_train[f], y_train[f], train_mask[f], placeholders)
        feed_dict.update({placeholders['dropout']: FLAGS.dropout})

    # Training step
        outs = sess.run([model.opt_op, model.loss, model.layers[0].embedding, model.layers[1].embedding], feed_dict=feed_dict)
        loss += outs[1]
    # Validation
    print(loss)
print("Optimization Finished!")

0
26641.125012524426
1
25561.140921447426
2
25425.18307053484
3
25374.217676702887
4
25347.114838730544
5
25386.520625675097
6
25341.891068344936
7
25363.193473713472
8
25328.426094166003
9
25292.370333753526
Optimization Finished!


### Save Trained Model

In [ ]:
!mkdir /content/drive/MyDrive/Docbank/final_parsing2_768_gcn_trained_model_updated

In [ ]:
saver = tf.train.Saver()
saver.save(sess, '/content/drive/MyDrive/Docbank/final_parsing2_768_gcn_trained_model_updated/parsing2_gcn_768')

'/content/drive/MyDrive/Docbank/final_parsing2_768_gcn_trained_model_updated/parsing2_gcn_768'

## Load Trained Model

In [ ]:
ckpt = tf.train.get_checkpoint_state("/content/drive/MyDrive/Docbank/final_parsing2_768_gcn_trained_model_updated")
output_dic_test = {}
with tf.Session() as sess:
  saver=tf.train.import_meta_graph('/content/drive/MyDrive/Docbank/final_parsing2_768_gcn_trained_model_updated/parsing2_gcn_768.meta')
  sess.run(tf.global_variables_initializer())
  saver.restore(sess, ckpt.all_model_checkpoint_paths[0])
  for f in features_train:
    cost, emb1, emb2 = evaluate(features_train[f], support_train[f], y_train[f], train_mask[f], placeholders)
    output_dic_test[f] = emb1
print("Optimization Finished!")
print(ckpt)


INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/Docbank/final_parsing2_768_gcn_trained_model_updated/parsing2_gcn_768
Optimization Finished!
model_checkpoint_path: "/content/drive/MyDrive/Docbank/final_parsing2_768_gcn_trained_model_updated/parsing2_gcn_768"
all_model_checkpoint_paths: "/content/drive/MyDrive/Docbank/final_parsing2_768_gcn_trained_model_updated/parsing2_gcn_768"



In [ ]:
import pickle
import os 
path = '/content/drive/MyDrive/Docbank/gcn_trained_embs/'
file_name = 'train_gcn_parsing2_768_part2.pkl' 
with open(path+file_name,'wb') as file_object:
    pickle.dump(output_dic_test,file_object)

In [ ]:
for f in features_eval:
    cost, acc, pred, labels, emb1, emb2 = evaluate(
        features_eval[f], support_eval[f], y_val[f], val_mask[f], placeholders)
    output_dic_test[f] = emb1

In [ ]:
output_dic_train_bert = output_dic_train
output_dic_test_bert = output_dic_test

In [ ]:
for img in output_dic_train_bert:
  for i in range(len(train_list_dict[img]['objects'].keys())):
    train_list_dict[img]['objects'][str(i)]['gcn_pos_emb'] = output_dic_train_bert[img][i] 

for img in output_dic_test_bert:
  for i in range(len(eval_list_dict[img]['objects'].keys())):
    eval_list_dict[img]['objects'][str(i)]['gcn_pos_emb'] = output_dic_test_bert[img][i] 

In [ ]:
bert_list_train = []
bert_list_test = []

In [ ]:
for img in train_list_dict:
  for obj in train_list_dict[img]['objects']:
    c_obj = train_list_dict[img]['objects'][obj]
    bert_list_train.append(c_obj['gcn_pos_emb'])

for img in eval_list_dict:
  for obj in eval_list_dict[img]['objects']:
    c_obj = eval_list_dict[img]['objects'][obj]
    bert_list_test.append(c_obj['gcn_pos_emb'])

In [ ]:
print(len(bert_list_train[0]))
print(len(bert_list_test))

768
2332


In [ ]:
df_train = pd.read_pickle('/content/drive/MyDrive/funsd/funsd_object_gcn_visual_density_bert_base_gcn_bert_base_train_parsing1_parsing2_pos.pkl')
df_test = pd.read_pickle('/content/drive/MyDrive/funsd/funsd_object_gcn_visual_density_bert_base_gcn_bert_base_test_parsing1_parsing2_pos.pkl')

In [ ]:
df_train['gcn_pos_emb'] = bert_list_train
df_test['gcn_pos_emb'] = bert_list_test

In [ ]:
df_train.to_pickle('/content/drive/MyDrive/funsd/funsd_object_gcn_visual_density_bert_base_gcn_bert_base_train_parsing1_parsing2_pos.pkl')
df_test.to_pickle('/content/drive/MyDrive/funsd/funsd_object_gcn_visual_density_bert_base_gcn_bert_base_test_parsing1_parsing2_pos.pkl')


In [ ]:
df_train.head()

,text,label,visual_feature,density,bert_base_cls,gcn_bert_base,level1_parse_emb,level2_parse_emb,near_density,near_visual_feature,gcn_near_char_density,gcn_near_char_number,gcn_near_token_density,gcn_near_token_number,gcn_parsing1,gcn_parsing2,pos_tag_emb,gcn_pos_emb
0,FROM:,0,"[-0.14605296, -0.16459094, -0.05297, -0.055089...","[-0.055626415, -0.04288777, -0.053692736, -0.0...","[-1.1602972745895386, 0.40634241700172424, -0....","[-5.773429, -1.0417114, -2.1828141, -0.788962,...","[0.2709057927131653, -0.962605893611908, 0.806...","[0.0, 1.0, 1.0, 1.0, -1.0, 1.0, 0.0, -0.0, -1....","[-0.038513724, -0.057523392, -0.07019832, -0.0...","[-0.032794982, -0.19511, -0.05077531, -0.04878...","[-0.054437697, -0.017164217, -0.046604343, 0.1...","[-0.05678776, -0.05876425, -0.058096003, -0.05...","[-0.03138002, -0.040639125, -0.0595022, -0.018...","[-0.053816292, -0.05289785, -0.054027267, -0.0...","[-0.033245698, -0.39197296, -0.06698716, -0.07...","[-2.3100514, -2.2200646, -1.0136094, -0.759668...","[0.0, -0.0, -1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0...","[-1.0204778, -0.47903004, -0.32504275, -0.2633..."
1,TO:,0,"[-0.07444859, -0.14759289, -0.05498426, -0.063...","[-0.059109252, -0.044888303, -0.057427548, -0....","[0.34755223989486694, 0.48073655366897583, -0....","[-6.957513, -1.1187, -2.230215, -1.0413656, -5...","[0.2709057927131653, -0.962605893611908, 0.806...","[0.0, 1.0, 1.0, 1.0, -1.0, 1.0, 0.0, -0.0, -1....","[-0.035946757, -0.05469693, -0.068453014, -0.0...","[0.008935843, -0.10750139, -0.051942326, -0.03...","[-0.05660252, -0.018605266, -0.045585666, 0.13...","[-0.049016792, -0.04999421, -0.053639263, -0.0...","[-0.030075125, -0.037472975, -0.05421398, -0.0...","[-0.061436642, -0.060031842, -0.06345818, -0.0...","[-0.035597026, -0.4854542, -0.0648092, -0.0764...","[-1.9864786, -2.022625, -1.0470014, -0.695816,...","[0.0, -0.0, -1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0...","[-0.9525763, -0.4266817, -0.32094085, -0.21244..."
2,CC:,0,"[-0.16252093, -0.12962683, -0.057510987, -0.06...","[-0.061404005, -0.046756633, -0.061196566, -0....","[-0.03836097568273544, 0.68309086561203, -0.21...","[-2.7018356, -1.2121015, -2.5539374, -1.168015...","[0.2709057927131653, -0.962605893611908, 0.806...","[0.0, 1.0, 1.0, 1.0, -1.0, 1.0, -0.0, 0.0, -1....","[-0.03539413, -0.047618706, -0.06595866, -0.04...","[0.084996946, -0.19689481, -0.04419114, -0.031...","[-0.047469404, -0.02167414, -0.04536049, 0.141...","[-0.044261906, -0.050114125, -0.049546476, -0....","[-0.025893634, -0.034434173, -0.056472085, -0....","[-0.056683403, -0.05927422, -0.059557024, -0.0...","[-0.037243284, -0.45954823, -0.06369393, -0.07...","[-2.0765593, -2.0653248, -1.0800002, -0.658574...","[0.0, -0.0, -1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0...","[-1.0286398, -0.4606863, -0.36516607, -0.27028..."
3,MEDIA,0,"[-0.016628355, -0.039290257, -0.051831502, -0....","[-0.0576571, -0.04908401, -0.05640934, -0.0417...","[-0.11887850612401962, 0.06347160786390305, 0....","[-3.6020103, -0.9338926, -2.335067, -0.8474312...","[0.2709057927131653, -0.962605893611908, 0.806...","[0.0, 1.0, 1.0, 1.0, -1.0, 1.0, -0.0, 0.0, -1....","[-0.035100132, -0.052675277, -0.065407366, -0....","[-0.04977132, -0.109212466, -0.04089707, -0.03...","[-0.05202429, -0.012842626, -0.045560665, 0.11...","[-0.057137825, -0.05482267, -0.06060242, -0.05...","[-0.028566573, -0.040235694, -0.051490035, -0....","[-0.052258953, -0.055718645, -0.056104325, -0....","[-0.039581835, -0.48810416, -0.0642253, -0.082...","[-2.1292121, -2.0249276, -0.92865527, -0.67442...","[0.0, -0.0, -1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0...","[-1.0143833, -0.45260423, -0.36988825, -0.2379..."
4,SPACE/COLOR,0,"[-0.042129744, -0.050594453, -0.06379194, -0.0...","[-0.06207407, -0.045576364, -0.057492614, -0.0...","[-0.25426730513572693, 0.5315051674842834, -0....","[-5.1081696, -1.3020573, -1.6366057, -0.604607...","[0.2709057927131653, -0.962605893611908, 0.806...","[0.0, 1.0, 1.0, 1.0, -1.0, 1.0, 0.0, 0.0, -1.0...","[-0.034166675, -0.04868983, -0.06372996